In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install torch torchvision open-clip-torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.6 MB/s eta 0:00:00


In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image, ImageFile

# Handle truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [10]:
train_dir = '/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train'
val_dir = '/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val'
test_dir = '/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test'

# Preprocessing pipeline for CLIP
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match CLIP input size
    transforms.ToTensor(),         # Convert image to tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize values
])

# Custom dataset to skip problematic images
class SafeImageFolder(ImageFolder):
    def __getitem__(self, index):
        try:
            return super(SafeImageFolder, self).__getitem__(index)
        except Exception as e:
            print(f"Error loading image at index {index}: {e}")
            return None

# Data loaders with custom collate function to skip None
def custom_collate(batch):
    return [b for b in batch if b is not None]

train_dataset = SafeImageFolder(root=train_dir, transform=preprocess)
val_dataset = SafeImageFolder(root=val_dir, transform=preprocess)
test_dataset = SafeImageFolder(root=test_dir, transform=preprocess)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=custom_collate)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=custom_collate)

# Verify dataset loading
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Training samples: 9246
Validation samples: 829
Test samples: 506


In [12]:
import open_clip

device = 'cuda' if torch.cuda.is_available() else 'cpu'
clip_model, _, _ = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
clip_model.eval().to(device)  # Set CLIP to evaluation mode

# Add classification head
class CLIPBinaryClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=2):
        super(CLIPBinaryClassifier, self).__init__()
        self.clip_model = clip_model
        self.clip_model.eval()  # Freeze CLIP
        self.classifier = nn.Linear(self.clip_model.visual.output_dim, num_classes)

    def forward(self, images):
        with torch.no_grad():  # Do not backpropagate through CLIP's encoder
            features = self.clip_model.encode_image(images)
        return self.classifier(features)

model = CLIPBinaryClassifier(clip_model).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-4)

In [13]:
def train_model(model, train_loader, val_loader, epochs=5):
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            images, labels = zip(*batch)  # Unpack images and labels
            images = torch.stack(images).to(device)
            labels = torch.tensor(labels).to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")
        validate_model(model, val_loader)

In [14]:
def validate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_loader:
            images, labels = zip(*batch)
            images = torch.stack(images).to(device)
            labels = torch.tensor(labels).to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {100 * correct / total:.2f}%")

# Training the model
train_model(model, train_loader, val_loader, epochs=10)

Epoch [1/10], Loss: 0.6727


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Validation Accuracy: 68.76%
Epoch [2/10], Loss: 0.6389
Validation Accuracy: 71.89%
Epoch [3/10], Loss: 0.6134
Validation Accuracy: 75.27%
Epoch [4/10], Loss: 0.5927
Validation Accuracy: 76.24%
Epoch [5/10], Loss: 0.5753
Validation Accuracy: 75.03%
Epoch [6/10], Loss: 0.5603
Validation Accuracy: 77.68%
Epoch [7/10], Loss: 0.5467
Validation Accuracy: 77.68%
Epoch [8/10], Loss: 0.5349
Validation Accuracy: 77.93%
Epoch [9/10], Loss: 0.5240
Validation Accuracy: 77.20%
Epoch [10/10], Loss: 0.5141
Validation Accuracy: 77.32%


In [15]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in test_loader:
            images, labels = zip(*batch)
            images = torch.stack(images).to(device)
            labels = torch.tensor(labels).to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

# Evaluate on the test dataset
test_model(model, test_loader)

Test Accuracy: 68.77%
